In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
import time

import os
import fnmatch

Using TensorFlow backend.


In [131]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [3]:
x_train = np.zeros((40479,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:54<00:00, 748.67it/s]

(40479, 32, 32, 3)
(40479, 17)


In [4]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean

In [5]:
#weather classifier (last four labels - mutually exclusive)
x_train, x_val, y_train_w, y_val_w = train_test_split(x_train,y_train[:,-4:],test_size=0.1)
print(x_train.shape)
print(y_train_w.shape)
print(x_val.shape)
print(y_val_w.shape)

(36431, 32, 32, 3)
(36431, 4)
(4048, 32, 32, 3)
(4048, 4)


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='softmax'))

In [9]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
20s - loss: 0.5161 - acc: 0.8233 - val_loss: 0.7777 - val_acc: 0.7485
Epoch 2/10
18s - loss: 0.3835 - acc: 0.8670 - val_loss: 0.4103 - val_acc: 0.8488
Epoch 3/10
19s - loss: 0.3351 - acc: 0.8835 - val_loss: 0.3417 - val_acc: 0.8841
Epoch 4/10
19s - loss: 0.3190 - acc: 0.8878 - val_loss: 0.2958 - val_acc: 0.8977
Epoch 5/10
19s - loss: 0.3096 - acc: 0.8910 - val_loss: 0.3509 - val_acc: 0.8718
Epoch 6/10
18s - loss: 0.2947 - acc: 0.8944 - val_loss: 0.3337 - val_acc: 0.8832
Epoch 7/10
19s - loss: 0.2989 - acc: 0.8960 - val_loss: 0.2769 - val_acc: 0.9049
Epoch 8/10
18s - loss: 0.2855 - acc: 0.8992 - val_loss: 0.3117 - val_acc: 0.8962
Epoch 9/10
19s - loss: 0.2837 - acc: 0.8988 - val_loss: 0.2519 - val_acc: 0.9091
Epoch 10/10
19s - loss: 0.2814 - acc: 0.8990 - val_loss: 0.2595 - val_acc: 0.9059


In [10]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.928756705251
thresh: 0.1 	F2 score: 0.932237260023
thresh: 0.15 	F2 score: 0.932825439959
thresh: 0.2 	F2 score: 0.930919736966
thresh: 0.25 	F2 score: 0.929294889893
thresh: 0.3 	F2 score: 0.924736495389
thresh: 0.35 	F2 score: 0.919631093544


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [11]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
19s - loss: 0.2683 - acc: 0.9037 - val_loss: 0.2387 - val_acc: 0.9160
Epoch 2/10
19s - loss: 0.2567 - acc: 0.9077 - val_loss: 0.2232 - val_acc: 0.9237
Epoch 3/10
19s - loss: 0.2521 - acc: 0.9103 - val_loss: 0.2323 - val_acc: 0.9209
Epoch 4/10
19s - loss: 0.2535 - acc: 0.9094 - val_loss: 0.2954 - val_acc: 0.8980
Epoch 5/10
19s - loss: 0.2490 - acc: 0.9103 - val_loss: 0.2275 - val_acc: 0.9172


In [12]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.928659655562
thresh: 0.1 	F2 score: 0.937886728308
thresh: 0.15 	F2 score: 0.940120341615
thresh: 0.2 	F2 score: 0.9384175607
thresh: 0.25 	F2 score: 0.938699887069
thresh: 0.3 	F2 score: 0.935123753059
thresh: 0.35 	F2 score: 0.930212450593


In [13]:
#continue with reduced learning rate
model.compile(loss='categorical_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
19s - loss: 0.2317 - acc: 0.9170 - val_loss: 0.2031 - val_acc: 0.9227
Epoch 2/10
19s - loss: 0.2295 - acc: 0.9173 - val_loss: 0.2121 - val_acc: 0.9256
Epoch 3/10
19s - loss: 0.2273 - acc: 0.9189 - val_loss: 0.1957 - val_acc: 0.9276
Epoch 4/10
19s - loss: 0.2274 - acc: 0.9190 - val_loss: 0.2070 - val_acc: 0.9212
Epoch 5/10
19s - loss: 0.2245 - acc: 0.9193 - val_loss: 0.2017 - val_acc: 0.9266
Epoch 6/10
18s - loss: 0.2226 - acc: 0.9198 - val_loss: 0.1965 - val_acc: 0.9286


In [14]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.936632434594
thresh: 0.1 	F2 score: 0.946532973367
thresh: 0.15 	F2 score: 0.947222614342
thresh: 0.2 	F2 score: 0.947787267081
thresh: 0.25 	F2 score: 0.946904997177
thresh: 0.3 	F2 score: 0.944664031621
thresh: 0.35 	F2 score: 0.940011528327


In [15]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_w, batch_size = 128), validation_data=(x_val, y_val_w),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
19s - loss: 0.2179 - acc: 0.9208 - val_loss: 0.1895 - val_acc: 0.9264
Epoch 2/10
18s - loss: 0.2140 - acc: 0.9216 - val_loss: 0.1958 - val_acc: 0.9291
Epoch 3/10
18s - loss: 0.2187 - acc: 0.9208 - val_loss: 0.2020 - val_acc: 0.9249
Epoch 4/10
19s - loss: 0.2174 - acc: 0.9217 - val_loss: 0.1877 - val_acc: 0.9298
Epoch 5/10
19s - loss: 0.2155 - acc: 0.9222 - val_loss: 0.1885 - val_acc: 0.9289
Epoch 6/10
18s - loss: 0.2159 - acc: 0.9219 - val_loss: 0.1935 - val_acc: 0.9286
Epoch 7/10
18s - loss: 0.2145 - acc: 0.9213 - val_loss: 0.1977 - val_acc: 0.9276


In [16]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_w, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.93593691182
thresh: 0.1 	F2 score: 0.946180065406
thresh: 0.15 	F2 score: 0.94889304536
thresh: 0.2 	F2 score: 0.948134293243
thresh: 0.25 	F2 score: 0.945922736684
thresh: 0.3 	F2 score: 0.943058300395
thresh: 0.35 	F2 score: 0.940876152833


In [17]:
model.save("simple_weather")

In [18]:
x_val = []
x_train = []
x_train = np.zeros((40479,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:53<00:00, 751.10it/s]

(40479, 32, 32, 3)
(40479, 17)


In [19]:
y_train_2 = np.zeros((y_train.shape[0],7))
y_train_2[:,1:] = y_train[:,7:13]
y_train_2[:,0] = (np.sum(y_train[:,:7],axis=1)>0)
y_train_2 = np.array(y_train_2,np.uint8)

In [20]:
print(y_train[100,:])
print(y_train_2[100,:])

[0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0]
[1 0 0 0 0 1 0]


In [21]:
x_train -= train_mean
x_train, x_val, y_train_2, y_val_2 = train_test_split(x_train,y_train_2,test_size=0.1)
print(x_train.shape)
print(y_train_2.shape)
print(x_val.shape)
print(y_val_2.shape)

(36431, 32, 32, 3)
(36431, 7)
(4048, 32, 32, 3)
(4048, 7)


In [41]:
#model for the more common classes + 1 class for others
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(32, 32, 3)))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(48, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(64, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Conv2D(128, (3, 3), padding = 'same'))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(2048))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(1024))
model1.add(BatchNormalization())
model1.add(Activation('relu'))
model1.add(Dense(7, activation='sigmoid'))

In [42]:
model1.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
53s - loss: 0.2913 - acc: 0.8805 - val_loss: 0.6006 - val_acc: 0.7722
Epoch 2/10
49s - loss: 0.2506 - acc: 0.8973 - val_loss: 0.2989 - val_acc: 0.8820
Epoch 3/10
49s - loss: 0.2372 - acc: 0.9043 - val_loss: 0.2562 - val_acc: 0.8950
Epoch 4/10
49s - loss: 0.2293 - acc: 0.9074 - val_loss: 0.2509 - val_acc: 0.8949
Epoch 5/10
49s - loss: 0.2238 - acc: 0.9102 - val_loss: 0.2646 - val_acc: 0.8841
Epoch 6/10
49s - loss: 0.2196 - acc: 0.9115 - val_loss: 0.2200 - val_acc: 0.9107
Epoch 7/10
49s - loss: 0.2158 - acc: 0.9129 - val_loss: 0.2580 - val_acc: 0.9026
Epoch 8/10
49s - loss: 0.2141 - acc: 0.9136 - val_loss: 0.2244 - val_acc: 0.9078
Epoch 9/10
49s - loss: 0.2122 - acc: 0.9151 - val_loss: 0.2196 - val_acc: 0.9100
Epoch 10/10
49s - loss: 0.2101 - acc: 0.9158 - val_loss: 0.2114 - val_acc: 0.9155


In [43]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.832920726117
thresh: 0.1 	F2 score: 0.85047125381
thresh: 0.15 	F2 score: 0.853083601714
thresh: 0.2 	F2 score: 0.85218122332
thresh: 0.25 	F2 score: 0.850658866404
thresh: 0.3 	F2 score: 0.847377603952
thresh: 0.35 	F2 score: 0.841522262036


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [47]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
53s - loss: 0.1891 - acc: 0.9245 - val_loss: 0.2002 - val_acc: 0.9154
Epoch 2/10
49s - loss: 0.1880 - acc: 0.9246 - val_loss: 0.1918 - val_acc: 0.9219
Epoch 3/10
49s - loss: 0.1873 - acc: 0.9251 - val_loss: 0.1936 - val_acc: 0.9185
Epoch 4/10
49s - loss: 0.1867 - acc: 0.9254 - val_loss: 0.1867 - val_acc: 0.9240
Epoch 5/10
49s - loss: 0.1865 - acc: 0.9256 - val_loss: 0.1944 - val_acc: 0.9206
Epoch 6/10
49s - loss: 0.1859 - acc: 0.9254 - val_loss: 0.1907 - val_acc: 0.9213
Epoch 7/10
49s - loss: 0.1851 - acc: 0.9261 - val_loss: 0.1963 - val_acc: 0.9192


In [48]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.839125476441
thresh: 0.1 	F2 score: 0.860659832221
thresh: 0.15 	F2 score: 0.863039656215
thresh: 0.2 	F2 score: 0.861837464378
thresh: 0.25 	F2 score: 0.856139575274
thresh: 0.3 	F2 score: 0.851963944031
thresh: 0.35 	F2 score: 0.845136486486


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [50]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
53s - loss: 0.1765 - acc: 0.9292 - val_loss: 0.1782 - val_acc: 0.9275
Epoch 2/10
49s - loss: 0.1753 - acc: 0.9301 - val_loss: 0.1778 - val_acc: 0.9285
Epoch 3/10
49s - loss: 0.1748 - acc: 0.9296 - val_loss: 0.1782 - val_acc: 0.9275
Epoch 4/10
49s - loss: 0.1740 - acc: 0.9300 - val_loss: 0.1790 - val_acc: 0.9276
Epoch 5/10
49s - loss: 0.1739 - acc: 0.9299 - val_loss: 0.1767 - val_acc: 0.9285
Epoch 6/10
49s - loss: 0.1737 - acc: 0.9304 - val_loss: 0.1772 - val_acc: 0.9272
Epoch 7/10
49s - loss: 0.1740 - acc: 0.9297 - val_loss: 0.1786 - val_acc: 0.9262
Epoch 8/10
49s - loss: 0.1727 - acc: 0.9308 - val_loss: 0.1777 - val_acc: 0.9281


In [51]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.850234659435
thresh: 0.1 	F2 score: 0.865652488317
thresh: 0.15 	F2 score: 0.869269455869
thresh: 0.2 	F2 score: 0.870728967268
thresh: 0.25 	F2 score: 0.869436172073
thresh: 0.3 	F2 score: 0.866010172428
thresh: 0.35 	F2 score: 0.861534247248


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [52]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  verbose=2, epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
53s - loss: 0.1708 - acc: 0.9313 - val_loss: 0.1750 - val_acc: 0.9286
Epoch 2/10
49s - loss: 0.1702 - acc: 0.9317 - val_loss: 0.1768 - val_acc: 0.9284
Epoch 3/10
50s - loss: 0.1702 - acc: 0.9318 - val_loss: 0.1768 - val_acc: 0.9265
Epoch 4/10
49s - loss: 0.1704 - acc: 0.9319 - val_loss: 0.1750 - val_acc: 0.9280
Epoch 5/10
49s - loss: 0.1694 - acc: 0.9318 - val_loss: 0.1769 - val_acc: 0.9274
Epoch 6/10
49s - loss: 0.1695 - acc: 0.9319 - val_loss: 0.1760 - val_acc: 0.9273
Epoch 7/10
49s - loss: 0.1690 - acc: 0.9321 - val_loss: 0.1756 - val_acc: 0.9269


In [53]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.849078641887
thresh: 0.1 	F2 score: 0.866140733274
thresh: 0.15 	F2 score: 0.871982758094
thresh: 0.2 	F2 score: 0.871557388887
thresh: 0.25 	F2 score: 0.870158703687
thresh: 0.3 	F2 score: 0.865118892491
thresh: 0.35 	F2 score: 0.86009692893


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [54]:
y_pred = model1.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score        

thresh: 0.05 	F2 score: 0.849078641887
thresh: 0.1 	F2 score: 0.866140733274
thresh: 0.15 	F2 score: 0.871982758094
thresh: 0.2 	F2 score: 0.871557388887
thresh: 0.25 	F2 score: 0.870158703687
thresh: 0.3 	F2 score: 0.865118892491
thresh: 0.35 	F2 score: 0.86009692893


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


In [55]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_2,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=['others']+labels[7:13])

,FN,FP,TN,TP,Total
others,85,256,3584,123,208
cultivation,91,471,3124,362,453
habitation,68,310,3363,307,375
road,56,479,2748,765,821
agriculture,58,485,2337,1168,1226
water,110,491,2782,665,775
primary,14,130,149,3755,3769


In [56]:
num_rare = np.sum(np.sum(y_train[:,:7],axis=1)>0)
print(num_rare)

2180


In [105]:
model1.save("simple_major")

In [58]:
x_val = []
x_train = []
x_train = np.zeros((num_rare,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    if(np.sum(targets[:7])>0):
        x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
        i+=1
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:53<00:00, 759.63it/s]

(2180, 32, 32, 3)
(2180, 17)


In [59]:
x_train -= train_mean
x_train, x_val, y_train_3, y_val_3 = train_test_split(x_train,y_train[:,:7],test_size=0.1)
print(x_train.shape)
print(y_train_3.shape)
print(x_val.shape)
print(y_val_3.shape)

(1962, 32, 32, 3)
(1962, 7)
(218, 32, 32, 3)
(218, 7)


In [106]:
#model for the rarer classes
from keras.models import load_model
model2 = load_model("simple_major")
for layer in model2.layers[:-1]:
    layer.trainable = False
model2.layers[-1].trainable = True

In [107]:
model2.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  verbose=2, epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )#more steps per epoch to compensate for fewer images

Epoch 1/10
18s - loss: 0.6104 - acc: 0.8219 - val_loss: 0.2997 - val_acc: 0.8919
Epoch 2/10
9s - loss: 0.2733 - acc: 0.8929 - val_loss: 0.2459 - val_acc: 0.9024
Epoch 3/10
9s - loss: 0.2482 - acc: 0.9014 - val_loss: 0.2360 - val_acc: 0.9096
Epoch 4/10
9s - loss: 0.2396 - acc: 0.9028 - val_loss: 0.2318 - val_acc: 0.9050
Epoch 5/10
9s - loss: 0.2326 - acc: 0.9043 - val_loss: 0.2347 - val_acc: 0.9056
Epoch 6/10
9s - loss: 0.2273 - acc: 0.9067 - val_loss: 0.2280 - val_acc: 0.9083
Epoch 7/10
8s - loss: 0.2239 - acc: 0.9068 - val_loss: 0.2282 - val_acc: 0.9083
Epoch 8/10
9s - loss: 0.2196 - acc: 0.9092 - val_loss: 0.2260 - val_acc: 0.9056
Epoch 9/10
9s - loss: 0.2188 - acc: 0.9094 - val_loss: 0.2323 - val_acc: 0.9004
Epoch 10/10
9s - loss: 0.2164 - acc: 0.9097 - val_loss: 0.2255 - val_acc: 0.9043


In [108]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.71551565789
thresh: 0.1 	F2 score: 0.727870944041
thresh: 0.15 	F2 score: 0.738574307955
thresh: 0.2 	F2 score: 0.754034701282
thresh: 0.25 	F2 score: 0.746849480794
thresh: 0.3 	F2 score: 0.715722351043
thresh: 0.35 	F2 score: 0.684194755296


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [109]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  verbose=2, epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
18s - loss: 0.2144 - acc: 0.9105 - val_loss: 0.2261 - val_acc: 0.9050
Epoch 2/10
9s - loss: 0.2126 - acc: 0.9108 - val_loss: 0.2252 - val_acc: 0.9050
Epoch 3/10
9s - loss: 0.2119 - acc: 0.9113 - val_loss: 0.2230 - val_acc: 0.9063
Epoch 4/10
9s - loss: 0.2120 - acc: 0.9105 - val_loss: 0.2273 - val_acc: 0.9017
Epoch 5/10
9s - loss: 0.2091 - acc: 0.9123 - val_loss: 0.2237 - val_acc: 0.9069
Epoch 6/10
9s - loss: 0.2086 - acc: 0.9119 - val_loss: 0.2238 - val_acc: 0.9063


In [110]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.723449365365
thresh: 0.1 	F2 score: 0.734549376465
thresh: 0.15 	F2 score: 0.74807321883
thresh: 0.2 	F2 score: 0.749192703321
thresh: 0.25 	F2 score: 0.736384410696
thresh: 0.3 	F2 score: 0.701706041156
thresh: 0.35 	F2 score: 0.665846131442


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [111]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  verbose=2, epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
17s - loss: 0.2087 - acc: 0.9123 - val_loss: 0.2252 - val_acc: 0.9030
Epoch 2/10
9s - loss: 0.2079 - acc: 0.9120 - val_loss: 0.2248 - val_acc: 0.9043
Epoch 3/10
9s - loss: 0.2086 - acc: 0.9117 - val_loss: 0.2238 - val_acc: 0.9050
Epoch 4/10
9s - loss: 0.2078 - acc: 0.9124 - val_loss: 0.2242 - val_acc: 0.9050
Epoch 5/10
9s - loss: 0.2073 - acc: 0.9120 - val_loss: 0.2244 - val_acc: 0.9043
Epoch 6/10
9s - loss: 0.2068 - acc: 0.9127 - val_loss: 0.2243 - val_acc: 0.9043


In [112]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.725217856812
thresh: 0.1 	F2 score: 0.734775259145
thresh: 0.15 	F2 score: 0.741195539934
thresh: 0.2 	F2 score: 0.744248106633
thresh: 0.25 	F2 score: 0.754478192552
thresh: 0.3 	F2 score: 0.709132865096
thresh: 0.35 	F2 score: 0.683721477299


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [117]:
for layer in model2.layers[-7:]:
    layer.trainable = True
model2.layers[-7:]

In [118]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  verbose=2, epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
19s - loss: 0.2017 - acc: 0.9149 - val_loss: 0.2204 - val_acc: 0.9050
Epoch 2/10
10s - loss: 0.1935 - acc: 0.9174 - val_loss: 0.2186 - val_acc: 0.9063
Epoch 3/10
10s - loss: 0.1868 - acc: 0.9202 - val_loss: 0.2164 - val_acc: 0.9050
Epoch 4/10
10s - loss: 0.1822 - acc: 0.9222 - val_loss: 0.2163 - val_acc: 0.9069
Epoch 5/10
10s - loss: 0.1767 - acc: 0.9248 - val_loss: 0.2147 - val_acc: 0.9102
Epoch 6/10
10s - loss: 0.1735 - acc: 0.9256 - val_loss: 0.2155 - val_acc: 0.9089
Epoch 7/10
10s - loss: 0.1673 - acc: 0.9287 - val_loss: 0.2157 - val_acc: 0.9122
Epoch 8/10
10s - loss: 0.1637 - acc: 0.9309 - val_loss: 0.2178 - val_acc: 0.9083


In [119]:
y_pred = model2.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score    

thresh: 0.05 	F2 score: 0.754225164489
thresh: 0.1 	F2 score: 0.759764254604
thresh: 0.15 	F2 score: 0.766460476323
thresh: 0.2 	F2 score: 0.765727646462
thresh: 0.25 	F2 score: 0.74368215882
thresh: 0.3 	F2 score: 0.719763417011
thresh: 0.35 	F2 score: 0.682592891308


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [121]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_3,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels[:7])

,FN,FP,TN,TP,Total
blow_down,5,14,195,4,9
bare_ground,8,57,64,89,97
conventional_mine,0,13,196,9,9
blooming,1,24,166,27,28
artisinal_mine,5,18,161,34,39
selective_logging,3,39,154,22,25
slash_burn,9,25,172,12,21


In [122]:
model2.save("simple_rare")

In [126]:
#F2 score on training set
x_val = []
x_train = []
x_train = np.zeros((40479,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'agriculture',
 'water',
 'primary',
 'partly_cloudy', 
 'cloudy',
 'clear',
 'haze',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:54<00:00, 737.27it/s]

(40479, 32, 32, 3)
(40479, 17)


In [127]:
x_train -= train_mean

In [128]:
y = model.predict(x_train,batch_size=128)
y1 = model1.predict(x_train,batch_size=128)
y2 = model2.predict(x_train,batch_size=128)

In [129]:
print(y.shape)
print(y1.shape)
print(y2.shape)

(40479, 4)
(40479, 7)
(40479, 7)


In [145]:
f2scorelist = []
for thresh in [0.05,0.1,0.15,0.2,0.25]:
    for thresh1 in [0.05,0.1,0.15,0.2,0.25]:
        for thresh2 in [0.05,0.1,0.15,0.2,0.25]:
            for thresh3 in [0.05,0.1,0.15,0.2,0.25]:
                y_pred = combine_predictions(x_train,y,y1,y2,thresh,thresh1,thresh2,thresh3)
                f2scorelist.append([thresh,thresh1,thresh2,thresh3,fbeta_score(y_train,y_pred , beta=2, average='samples')])

In [146]:
f2scorelist

[[0.05, 0.05, 0.05, 0.05, 0.88487940590272474],
 [0.05, 0.05, 0.05, 0.1, 0.89264315244330938],
 [0.05, 0.05, 0.05, 0.15, 0.89551484978156226],
 [0.05, 0.05, 0.05, 0.2, 0.8967728986636857],
 [0.05, 0.05, 0.05, 0.25, 0.89727301553856686],
 [0.05, 0.05, 0.1, 0.05, 0.88817938941534058],
 [0.05, 0.05, 0.1, 0.1, 0.89433195462957693],
 [0.05, 0.05, 0.1, 0.15, 0.89655755911311741],
 [0.05, 0.05, 0.1, 0.2, 0.89752277075214559],
 [0.05, 0.05, 0.1, 0.25, 0.89782702753133936],
 [0.05, 0.05, 0.15, 0.05, 0.89043734766218718],
 [0.05, 0.05, 0.15, 0.1, 0.89545733981985198],
 [0.05, 0.05, 0.15, 0.15, 0.89724853599324561],
 [0.05, 0.05, 0.15, 0.2, 0.89801332452067406],
 [0.05, 0.05, 0.15, 0.25, 0.89816409480216275],
 [0.05, 0.05, 0.2, 0.05, 0.89197129822226406],
 [0.05, 0.05, 0.2, 0.1, 0.89627251278860909],
 [0.05, 0.05, 0.2, 0.15, 0.89772610563198907],
 [0.05, 0.05, 0.2, 0.2, 0.89831469983139589],
 [0.05, 0.05, 0.2, 0.25, 0.89838100151963562],
 [0.05, 0.05, 0.25, 0.05, 0.89304248756717253],
 [0.05, 0.0

In [147]:
max([l[4] for l in f2scorelist])

0.92240080097623756

In [151]:
#Test set
x_train = []
x_val = []
x_test = np.zeros((61191,32,32,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

100%|██████████| 61191/61191 [04:38<00:00, 219.90it/s]

(61191, 32, 32, 3)


In [152]:
x_test -= train_mean

y = model.predict(x_test,batch_size=128)
y1 = model1.predict(x_test,batch_size=128)
y2 = model2.predict(x_test,batch_size=128)

In [156]:
thresh,thresh1,thresh2,thresh3 = 0.2, 0.2, 0.25, 0.25
y_pred = combine_predictions(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3)
print(y_pred.shape)

(61191, 17)


In [166]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_3net_1.csv', index=False)
#test set score:0.91479

In [167]:
subm

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,cultivation agriculture primary partly_cloudy ...
4,test_4,primary partly_cloudy cloudy
5,test_5,primary clear
6,test_6,cultivation road agriculture primary partly_cl...
7,test_7,habitation road agriculture primary clear
8,test_8,primary clear
9,test_9,cultivation agriculture primary clear haze


In [168]:
thresh,thresh1,thresh2,thresh3 = 0.2, 0.2, 0.25, 0.2
y_pred = combine_predictions(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3)
print(y_pred.shape)

(61191, 17)


In [169]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_3net_2.csv', index=False)
#test set score:0.91448

In [170]:
thresh,thresh1,thresh2,thresh3 = 0.2, 0.2, 0.2, 0.2
y_pred = combine_predictions(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3)
print(y_pred.shape)

(61191, 17)


In [171]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_3net_3.csv', index=False)
#test set score:0.91429

In [172]:
thresh,thresh1,thresh2,thresh3 = 0.25, 0.25, 0.25, 0.25
y_pred = combine_predictions(x_test,y,y1,y2,thresh,thresh1,thresh2,thresh3)
print(y_pred.shape)

(61191, 17)


In [173]:
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_3net_4.csv', index=False)
#test set score:0.91308